# Practical box size estimation from RFID interrogation traces using transfer learning 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Move to the directory where dataset_traces from RFID interrogation are placed
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/_BOX_SIZE')

%pwd

Mounted at /content/drive


'/content/drive/My Drive/Colab Notebooks/_BOX_SIZE'

In [ ]:
# Bibliotecas necesarias
import numpy as np
import random
import os

# Configuramos matplotlib
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

%load_ext tensorboard

# Semilla fija para reproducibilidad
np.random.seed(1)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas
import pickle

def change_distro(X,t,samples):  
    newX = np.zeros(shape=[samples.sum(),X.shape[1]])
    newt = np.zeros(shape=[samples.sum(),])
    numsamples = 0
    for i in range(t.max()+1):
        aux = X[t==i]
        newX[numsamples:numsamples+samples[i]] = aux[:samples[i]]
        newt[numsamples:numsamples+samples[i]] = i
        numsamples += samples[i]
    
    newt = newt.astype(int)
    return newX,newt

def readdataset(id, maxsize=0, debug=False):
    picklefile = f'{id}/data.pickle'

    try:
        with open(picklefile, 'rb') as f:
            X, t = pickle.load(f)
            print(f'Datos ya disponibles de {picklefile}')
    
    except:
        index = pandas.read_csv(f'{id}/index.{id}.txt')
        index["numframes"] = index.framelst-index.frameini+1
        maxframes = index.numframes.max()
        frames = pandas.read_csv(f'{id}/frames.{id}.txt')
    
        if maxsize>0:
            SAMPLES = maxsize
        else:
            SAMPLES = len(index)
            
        if debug:
            print(f'Samples: {SAMPLES}')
            
        X = np.zeros(shape=[SAMPLES, 23]) 
        t = np.zeros(shape=[SAMPLES, ]) 
    
        for i in range(SAMPLES):
            if maxsize>0:
                batch = random.randint(0,SAMPLES-1)
            else:
                batch = i
            tagsread = index.iloc[batch].tagsread
            readtime = index.iloc[batch].readtime
            numframes = index.iloc[batch].numframes
            framebase = int(index.iloc[batch].frameini) 
            aux = frames.loc[framebase:framebase+numframes-1].values
            X[i,0] = tagsread
            X[i,1] = readtime
            X[i,2] = numframes
            X[i,3:8] = aux.mean(axis=0)
            X[i,8:13] = aux.std(axis=0)
            X[i,13:18] = aux.min(axis=0)
            X[i,18:23] = aux.max(axis=0)
            t[i] = index.iloc[batch].boxoriginal
            #X[batch,3:3+numframes*5] = frames.loc[framebase:framebase+numframes-1].values.ravel()
            if (debug) and ((i+1)%5000)==0:
                print(f'i: {i+1}, batch: {batch}, frames: {numframes}, framebase: {framebase}, t: {t[i]}, aux: {aux}')
        
            t = t.astype(int)
   
        with open(picklefile, 'wb') as f:
            pickle.dump([X, t], f)
    
        if debug: 
            frames.head(-1)
            
    return X,t

In [ ]:
import keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

def evalmodel(X,t,id,info='Full',debug=False):  
    modelname = f'{id}/model{info}.h5'
    
    if info=='Tags':
        X = X[:,0:1]
    elif info=='Basic':
        X = X[:,0:3]
    # 'Full' -> Toda la info
    
    X_train, X_test, t_train, t_test =  train_test_split(X, t, test_size=0.8)
    
    standard_transformer = Pipeline(steps=[('standard', StandardScaler())])
    minmax_transformer = Pipeline(steps=[('minmax', MinMaxScaler())])
    preprocessor = ColumnTransformer(
        remainder='passthrough', #passthough features not listed
        transformers=[
            ('std', standard_transformer , []),#slice(0,X.shape[1])),
            ('mm', minmax_transformer , slice(0,X.shape[1]))
        ])
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)
    model = keras.models.Sequential([
        keras.layers.Dense(100, activation="relu", input_shape=X_train.shape[1:]),
        keras.layers.Dense(50, activation="tanh"),
        keras.layers.Dense(int(t.max())+1, activation="sigmoid")
    ])
    
    try: 
        model = keras.models.load_model(modelname) # cargar modelo
    except:
        early_stopping_cb = EarlyStopping(patience=100, restore_best_weights=True)
        checkpoint_cb = ModelCheckpoint(modelname, save_best_only=True)
        tensorboard_cb = TensorBoard(f'{id}/log', histogram_freq=1)
        model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        model.fit(X_train, t_train, epochs=50000
                  , validation_data=(X_test, t_test), batch_size=32, callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb], verbose=0)
        model.save(modelname)
    
    precision = model.evaluate(X_test,t_test)[1]
    #y_test = model.predict_classes(X_test)
    y = np.argmax(model.predict(X_processed),axis=1)
    conf = confusion_matrix(list(map(int,t_test)), y_test)
    
    return precision, conf

In [ ]:
escenarios={'CGG','CGM','CGP','CMG','CMM','CMP','CPG','CPM','CPP','IGG','IGM','IGP','IMG','IMM','IMP','IPG','IPM','IPP','SGG','SGM','SGP','SMG','SMM','SMP','SPG','SPM','SPP'}
infoset={'Full','Basic','Tags'}

for info in infoset:
    for escenario in escenarios:
        # %tensorboard --logdir f'{escenario}/log'   -> Descomentar para ver la evolucion del entrenamiento
        print(f'Escenario: {escenario}, Info: {info}')
        X, t = readdataset(escenario)
        precision, conf = evalmodel(X,t,escenario,info)
        print(f'Accuracy: {precision*100:0.2f}%\nConfussion matrix:\n{conf}')

Escenario: CPG, Info: Full
500/500 [==============================] - 1s 1ms/step - loss: 1.1272 - accuracy: 0.5162


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 51.62%
Confussion matrix:
[[931  49   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [206 679 130   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  4 124 699 168   3   0   0   0   0   0   0   0   0   0   0   0]
 [  0   4 183 603 188  27   1   0   0   0   0   0   0   0   0   0]
 [  0   0  21 278 619  59  31   4   0   0   0   0   0   0   0   0]
 [  0   0   0  14 154 657 142  27   9   9   0   0   0   0   0   0]
 [  0   0   0   2 100 206 354 215 120  33   1   0   0   0   0   0]
 [  0   0   0   0  12  73 202 275 249 164  10   0   0   0   0   0]
 [  0   0   0   0   4  12 112 160 437 170  37   2   0   0   0   0]
 [  0   0   0   0   0  11  15  64 158 535 260  13   0   0   1   0]
 [  0   0   0   0   0   0   0   7  50 259 518 106   3   1   2   0]
 [  0   0   0   0   0   0   0   0   0   8 114 576 206  40  58   5]
 [  0   0   0   0   0   0   0   0   0   2  36 326 311  87 191  66]
 [  0   0   0   0   0   0   0   0   0   0  13 175 219  74 318 180]
 [  0   0   0   0   0   0 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 67.73%
Confussion matrix:
[[960  17   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [222 790  10   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2  30 865 142   7   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 221 687  39   2   8   0   0   0   0   0   0   0   0   0]
 [  0   0   8  52 856  16  44   9   2   0   0   0   0   0   0   0]
 [  0   0   0   1  26 931   8  45  14   0   0   0   0   0   0   0]
 [  0   0   0  10  64   5 822  46   6  47   0   0   0   0   0   0]
 [  0   0   0   0  20  15  35 560 252  43  35   0   0   0   0   0]
 [  0   0   0   0   7  12  10 146 770   4  34  15   0   1   0   2]
 [  0   0   0   0   0   0  45  61  24 655 134   1   0   1   0   0]
 [  0   0   0   0   0   1   2  44  50  99 801  44  12   1   0   1]
 [  0   0   0   0   0   0   0   0  24   0  88 411 135 208   7  69]
 [  0   0   0   0   0   0   0   0   7   0  47 152 287 354  79  73]
 [  0   0   0   0   0   0   0   0   3   0  17 128 127 430 136 188]
 [  0   0   0   0   0   0 

...

Accuracy: 75.17%
Confussion matrix:
[[2018    5    0    0    0    0    0    0]
 [  31 1783  215    0    0    0    0    0]
 [   0  156 1557  288    1    0    0    0]
 [   0    1  410 1423  174    0    0    0]
 [   0    0    0  226 1411  298    3    0]
 [   0    0    0    0  276 1452  273   12]
 [   0    0    0    0    3  278  990  701]
 [   0    0    0    0    0   51  562 1402]]
Escenario: SMG, Info: Tags
Datos ya disponibles de SMG/data.pickle
500/500 [==============================] - 0s 804us/step - loss: 1.1973 - accuracy: 0.4767


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 47.67%
Confussion matrix:
[[1008    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [ 705  272   11    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  43  301  433  176    6    0    0    0    0    0    0    0    0    0
     0    0]
 [   0   20  177  528  253   21    0    0    0    0    0    0    0    0
     0    0]
 [   0    1   14  222  457  245   28    3    0    0    0    0    0    0
     0    0]
 [   0    0    1   47  284  445  202   32    3    1    0    0    0    0
     0    0]
 [   0    0    0    1   42  211  408  228   90   16    0    0    0    0
     0    0]
 [   0    0    0    0    3   52  206  294  271  123   13    0    0    0
     0    0]
 [   0    0    0    0    1   11  100  242  365  230   54    0    0    0
     0    0]
 [   0    0    0    0    0    1   11   61  207  395  258   13    0    0
     0    0]
 [   0    0    0    0    0    0    0    3   43  258  563  133    6    0
     0    0]
 [   0    0    0    0    0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 75.01%
Confussion matrix:
[[2074    0    0    0    0    0    0    0]
 [  85 1744  122    0    0    0    0    0]
 [   0  215 1365  367    2    0    0    0]
 [   0    4  338 1534  221    0    0    0]
 [   0    0    0  204 1539  281    0    0]
 [   0    0    0    2  285 1360  285   12]
 [   0    0    0    0    3  327 1053  635]
 [   0    0    0    0    0   60  587 1296]]
